In [1]:
import whisper
import soundfile as sf
import numpy as np
import librosa
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import sounddevice as sd
import os


c:\Users\HP\anaconda3\envs\signavatars\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Whisper (Tamil ASR)
print("Loading Whisper...")
whisper_model = whisper.load_model("medium").to("cuda")
print("✓ Whisper loaded!\n")

# mBART50 (Tamil → English)
print("Loading mBART50...")
model_name = "facebook/mbart-large-50-many-to-many-mmt"

tokenizer = MBart50TokenizerFast.from_pretrained(
    model_name,
    src_lang="ta_IN"
)

mbart_model = MBartForConditionalGeneration.from_pretrained(
    model_name
).to("cuda")

print("✓ mBART50 loaded!")


Loading Whisper...
✓ Whisper loaded!

Loading mBART50...


c:\Users\HP\anaconda3\envs\signavatars\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\HP\anaconda3\envs\signavatars\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--facebook--mbart-large-50-many-to-many-mmt. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or 

✓ mBART50 loaded!


In [3]:
def record_audio(filename="tamil.wav", duration=5, fs=16000):
    os.makedirs("audio", exist_ok=True)
    path = os.path.join("audio", filename)

    print(f"🎤 Recording for {duration} seconds...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype="float32")
    sd.wait()

    sf.write(path, audio, fs)
    print(f"Saved to: {path}")

    return path


In [4]:
def transcribe_tamil(audio_path):
    audio, sr = sf.read(audio_path)

    # mono
    if len(audio.shape) > 1:
        audio = audio.mean(axis=1)

    audio = audio.astype(np.float32)

    # resample to 16k
    if sr != 16000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)

    print("📝 Transcribing Tamil...")
    result = whisper_model.transcribe(audio, fp16=True)
    return result["text"]


In [5]:
def translate_tamil_to_english(tamil_text):
    print("🌐 Translating Tamil → English...")

    tokenizer.src_lang = "ta_IN"

    encoded = tokenizer(
        tamil_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    ).to("cuda")

    generated = mbart_model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"],
        num_beams=5,
        max_length=256
    )

    english = tokenizer.batch_decode(
        generated,
        skip_special_tokens=True
    )[0]

    return english


In [18]:
# Step 1: Record
audio_file = record_audio(duration=5)

# Step 2: Tamil STT
tamil_text = transcribe_tamil(audio_file)
print("\nTamil Transcription:")
print(tamil_text)

# Step 3: Translate
english_text = translate_tamil_to_english(tamil_text)
print("\nEnglish Translation:")
print(english_text)


🎤 Recording for 5 seconds...
Saved to: audio\tamil.wav
📝 Transcribing Tamil...

Tamil Transcription:
 நான் மருத்துவம் மணைக்கு செல்ல வேண்டும்
🌐 Translating Tamil → English...

English Translation:
I have to go to the medical center


In [10]:
!pip install stanza


   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.1/1.1 MB 2.8 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.1 MB 2.7 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.1 MB 2.2 MB/s eta 0:00:01
   -------------- ------------------------- 0.4/1.1 MB 1.9 MB/s eta 0:00:01
   ---------------------- ----------------- 0.6/1.1 MB 2.5 MB/s eta 0:00:01
   ---------------------------------- ----- 1.0/1.1 MB 3.4 MB/s eta 0:00:01
   ------------------------------------- -- 1.0/1.1 MB 3.5 MB/s eta 0:00:01
   -------------------------------------- - 1.1/1.1 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/608.4 kB ? eta -:--:--
   ---------------------------------- ---- 532.5/608.4 kB 11.1 MB/s eta 0:00:01
   ------------------------

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import stanza
stanza.download("en")


2026-01-26 19:59:37 INFO: Downloaded file to C:\Users\HP\stanza_resources\resources.json
2026-01-26 19:59:37 INFO: Downloading default packages for language: en (English) ...
2026-01-26 20:00:20 INFO: Downloaded file to C:\Users\HP\stanza_resources\en\default.zip
2026-01-26 20:00:23 INFO: Finished downloading models and saved to C:\Users\HP\stanza_resources


In [12]:
import stanza
nlp = stanza.Pipeline("en", processors="tokenize,pos,lemma,depparse")


2026-01-26 20:00:41 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2026-01-26 20:00:41 INFO: Downloaded file to C:\Users\HP\stanza_resources\resources.json
2026-01-26 20:00:41 WARNING: Language en package default expects mwt, which has been added
2026-01-26 20:00:42 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2026-01-26 20:00:42 INFO: Using device: cuda
2026-01-26 20:00:42 INFO: Loading: tokenize
2026-01-26 20:00:43 INFO: Loading: mwt
2026-01-26 20:00:44 INFO: Loading: pos
2026-01-26 20:00:49 INFO: Loading: lemma
2026-01-26 20:00:49 INFO: Loading: depparse
2026-01-26 20:00:50 INFO: Done loading p

In [13]:
def english_to_gloss_stanza(text):
    doc = nlp(text)
    sentence = doc.sentences[0]

    content_words = []
    verbs = []

    drop_pos = {"AUX", "ADP", "DET"}
    drop_words = {"the", "a", "an", "to"}

    for word in sentence.words:
        pos = word.upos
        lemma = word.lemma
        text = word.text

        # Skip junk words
        if pos in drop_pos or text.lower() in drop_words:
            continue

        # VERBS → base form + tense marking
        if pos == "VERB":
            lemma = lemma.upper()
            # tense
            if word.feats and "Tense=Past" in word.feats:
                lemma += "-PAST"
            verbs.append(lemma)
        else:
            content_words.append(text.upper())

    # SOV order → verbs moved to end
    gloss = content_words + verbs
    return " ".join(gloss)


In [19]:
gloss = english_to_gloss_stanza(english_text)
print(gloss)


I MEDICAL CENTER HAVE GO
